In [1]:
from scholarly import scholarly
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import numpy as np
from scholarly import scholarly
from typing import Union
from scholarly import ProxyGenerator
import requests
import Levenshtein

warnings.filterwarnings("ignore")
%load_ext jupyter_black

In [2]:
def levenshtein_similarity(str1, str2):
    distance = Levenshtein.distance(str1, str2)
    max_len = max(len(str1), len(str2))
    similarity = 1 - (distance / max_len)
    return similarity


# Example usage


def jaccard_similarity(str1, str2):
    set1 = set(str1.split())
    set2 = set(str2.split())
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    similarity = len(intersection) / len(union)
    return similarity


# Example usage
t1 = "A. Haroon Akram-Lodhi"
t2 = "Haroon"
similarity = levenshtein_similarity(t1, t2)
print(f"Levenshtein Similarity: {similarity}")

similarity = jaccard_similarity(t1, t2)
print(f"Jaccard Similarity: {similarity}")

Levenshtein Similarity: 0.2857142857142857
Jaccard Similarity: 0.3333333333333333


In [186]:
coauthors = []
family_name = "Shukla"  # "Skea"  # "SCHAEFFER"  # "riahi"  # "edenhofer"  # "jiang"  # "mignone"  # "heuberger"  # "williams" #"lee"   # "teletzke"  #  "kheshgi"  # "calvin"
given_name = "Priyadarshi"  # "Jim"  # "Roberto"  # "keywan"  # "ottmar"  # "bryan"  # "clara"  # "ellen" #"arthur"    ##"gary"  #  "haroon"  # "katherine"
family_name = family_name.lower()
given_name = given_name.lower()
base_url = "https://api.crossref.org/works"
start_year = 2004
end_year = 2024

In [3]:
princeton_df = pd.read_csv(
    "/Volumes/Samsung_T5/data/lobbying/princeton/postprocessed_filings.csv"
)

In [6]:
3.240260e04 / 6.397597e06

0.0050648079271013785

In [5]:
princeton_df[["definitely_ccs", "lobbying_activity_usd"]].groupby(
    "definitely_ccs"
).sum()

,lobbying_activity_usd
definitely_ccs,
0,6.397597e+06
1,3.240260e+04


In [25]:
# Parameters for the API request
params = {
    "query.author": given_name + " " + family_name,
    "filter": f"from-pub-date:{start_year}-01-01,until-pub-date:{end_year}-12-31",
    "rows": 1000,  # Number of results per page (max is 1000)
}

coauthors_set = set()  # To store unique co-authors


response = requests.get(base_url, params=params)
response.raise_for_status()  # Check for request errors
pub_list = response.json()["message"]["items"]
for pub in pub_list:
    authors = pub["author"]
    keep = True
    for a in authors:
        if "family" in a:
            if given_name in a["family"].lower():
                keep = False
            elif "given" in a:
                if (family_name in a["family"].lower()) and (
                    given_name not in a["given"].lower()
                ):
                    keep = False
                elif (family_name not in a["family"].lower()) and (
                    given_name in a["given"].lower()
                ):
                    keep = False
                elif family_name in a["given"].lower():
                    keep = False
                elif (a["family"].lower() == family_name) and (
                    given_name not in a["given"].lower()
                ):
                    keep = False
                elif (given_name in a["given"].lower()) and (
                    family_name not in a["family"].lower()
                ):
                    keep = False
    if keep:
        for author in authors:
            row_dict = {}
            row_dict["title"] = ""
            if "title" in pub:
                title = pub["title"][0]
            row_dict["title"] = title
            row_dict["ccs"] = 0
            for t in [
                "ccus",
                "ccs",
                "carbon capture",
                "carbon dioxide removal",
                "sequestration",
                "hydrogen",
                "carbon dioxide transport",
                "carbon dioxide capture",
                "co2 storage",
                "co2 transport",
            ]:
                if t in title.lower():
                    row_dict["ccs"] = 1
            row_dict["year"] = pub["published"]["date-parts"][0][0]
            row_dict["type"] = ""
            row_dict["doi"] = ""
            if "DOI" in pub:
                row_dict["doi"] = pub["DOI"]
            if "type" in pub:
                row_dict["type"] = pub["type"]
            row_dict["publisher"] = pub["publisher"]
            if "family" in author:
                if "given" in author:
                    author_full_name = (
                        author["given"].split(" ")[0] + " " + author["family"]
                    )
                else:
                    author_full_name = author["family"]
            else:
                author_full_name = author["name"]
            author_full_name = author_full_name.replace("‐", "-").replace(".", "")
            row_dict["author"] = author_full_name.title()
            row_dict["affiliation"] = ""
            if len(author["affiliation"]) > 0:
                row_dict["affiliation"] = author["affiliation"][0]["name"]
            coauthors.append(row_dict)

df = pd.DataFrame(coauthors)

In [26]:
df.ccs.value_counts()

ccs
0    114
Name: count, dtype: int64

In [27]:
df[["title", "author", "ccs"]].groupby("title").agg({"ccs": "max"}).ccs.value_counts()

ccs
0    36
Name: count, dtype: int64

In [28]:
tmp = (
    df[["title", "doi", "year", "ccs"]]
    .groupby("title")
    .agg({"ccs": "max", "doi": "first", "year": "first"})
)  # .ccs.value_counts()
tmp.loc[tmp.ccs == 1]

,ccs,doi,year
title,,,


In [29]:
df.groupby("author").agg(
    {
        "ccs": "sum",
        "title": "count",
        "year": "mean",
        "affiliation": lambda x: list(set(x)),
    }
).sort_values(by="title", ascending=False).head(60)

,ccs,title,year,affiliation
author,,,,
Priyadarshi Shukla,0,24,2017.000000,"[, Global Centre for Environment and Energy, A..."
Minal Pathak,0,8,2021.000000,"[, Ahmedabad University]"
Subash Dhar,0,6,2016.500000,[]
Shukla,0,6,2014.833333,[]
Vaibhav Chaturvedi,0,5,2013.400000,"[, Indian Institute of Management Ahmedabad I..."
Shivika Mittal,0,4,2019.500000,"[, Marine and Renewable Energy Ireland, Enviro..."
Purvi Vyas,0,3,2021.333333,"[, Ahmedabad University]"
Shaurya Patel,0,3,2021.333333,"[, Ahmedabad University]"
Gaurav Shukla,0,3,2022.000000,[]


In [154]:
authors = list(df.author.unique())
for a in authors:
    print(a)

Haroon S Kheshgi
Madhu Khanna
Haroon Kheshgi
Roger C Prince
David E Archer
Heleen De Coninck
John Kessels
Sergey Paltsev
Jennifer Morris
Howard Herzog
Steven J Smith
James A Edmonds
Shijie Shu
Atul K Jain
Michael Schlesinger
Joel Smith
John M Reilly
Tom Wilson
Matteo Muratori
Bryan Mignone
Haewon Mcjeon
Leon Clarke
Jae Edmonds
Karin Ritter
Robert Siveter
Miriam Lev-On
Theresa Shires
Angelo Gurgel
Bryan K Mignone
Steve Crookshank
Paulo Cunha
Arthur Lee
Leonard Bernstein
Erin Smith
Gary Teletzke
Jessica Farrell
Hans Thomann
Yang Song
William Landuyt
Felipe Feijoo
Corinne Hartin
Tzu-Shun Lin
Peter Lawrence
Atul Jain
Xiaojuan Yang
A David Mcguire
Wilfred Post
David Kicklighter
Oddvar Levang
Monica Linhares
Jaime Juez
Brigitte Poot
Charles J Vörösmarty
Henry Chen
Matthew Cervarich
Ximing Cai
Jaime Martin Juez
Page Kyle
Robert Link
Kali Benavides
Angelo C Gurgel
Bryan Chapman
Y-H Henry Chen
Matthew Mowers
Daniel Steinberg
Jordan Kearns
Jeffrey Palmer
Yen-Heng Henry Chen
Fredde Cappelen
Alain

In [57]:
for k in pub_list[40].keys():
    print(k)
    if isinstance(pub_list[40][k], dict):
        keys = pub_list[40][k].keys()
        for kk in keys:
            print(f" - {kk}")

indexed
 - date-parts
 - date-time
 - timestamp
reference-count
publisher
license
content-domain
 - domain
 - crossmark-restriction
short-container-title
published-print
 - date-parts
DOI
type
created
 - date-parts
 - date-time
 - timestamp
page
update-policy
source
is-referenced-by-count
special_numbering
title
prefix
volume
author
member
reference
container-title
language
link
deposited
 - date-parts
 - date-time
 - timestamp
score
resource
 - primary
issued
 - date-parts
references-count
alternative-id
URL
ISSN
issn-type
published
 - date-parts
assertion
article-number
